In [17]:
import cv2

import numpy as np
import time
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import FancyArrowPatch
# Use Agg backend for canvas
import matplotlib
matplotlib.use('Agg') 
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

from mpl_toolkits.mplot3d import proj3d

In [18]:
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs)

In [19]:
def estimatePoseDLT(p, P, K):
    """
    Estimates the pose of a camera using a set of 2D-3D correspondences  and a given camera matrix.
    Args:   p  [n x 2] array containing the undistorted coordinates of the 2D points
            P  [n x 3] array containing the 3D point positions
        K  [3 x 3] camera matrix
    Returns: a [3 x 4] projection matrix of the form: M_tilde = [R_tilde | alpha * t] 
            where R is a rotation matrix. M_tilde encodes the transformation that maps points from the world frame to the camera frame
    """
    # Convert 2D to normalized coordinates
    p_norm = (np.linalg.inv(K) @ np.c_[p, np.ones((p.shape[0], 1))].T).T

    # Build measurement matrix Q
    num_corners = p_norm.shape[0]
    Q = np.zeros((2*num_corners, 12))

    for i in range(num_corners):
        u = p_norm[i, 0]
        v = p_norm[i, 1]

        Q[2*i, 0:3] = P[i,:]
        Q[2*i, 3] = 1
        Q[2*i, 8:11] = -u * P[i,:]
        Q[2*i, 11] = -u
        
        Q[2*i+1, 4:7] = P[i,:]
        Q[2*i+1, 7] = 1
        Q[2*i+1, 8:11] = -v * P[i,:]
        Q[2*i+1, 11] = -v

    # Solve for Q.M_tilde = 0 subject to the constraint ||M_tilde||=1
    u, s, v = np.linalg.svd(Q, full_matrices=True)
    M_tilde = np.reshape(v.T[:,-1], (3,4));
    
    # Extract [R | t] with the correct scale
    if (np.linalg.det(M_tilde[:, :3]) < 0):
        M_tilde *= -1

    R = M_tilde[:, :3]

    # Find the closest orthogonal matrix to R
    # https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    u, s, v = np.linalg.svd(R);
    R_tilde = u @ v;

    # Normalization scheme using the Frobenius norm:
    # recover the unknown scale using the fact that R_tilde is a true rotation matrix
    alpha = np.linalg.norm(R_tilde, 'fro')/np.linalg.norm(R, 'fro');

    # Build M_tilde with the corrected rotation and scale
    M_tilde = np.c_[R_tilde, alpha * M_tilde[:,3]];
    
    return M_tilde

def reprojectPoints(P, M_tilde, K):
    """
    Reproject 3D points given a projection matrix
    Args:   P         [n x 3] coordinates of the 3d points in the world frame
            M_tilde   [3 x 4] projection matrix
            K         [3 x 3] camera matrix
    Returns [n x 2] coordinates of the reprojected 2d points
    """
    p_homo = (K @ M_tilde @ np.r_[P.T, np.ones((1, P.shape[0]))]).T
    return p_homo[:,:2]/p_homo[:,2,np.newaxis]


def drawCamera(ax, position, direction, length_scale = 1, head_size = 10, equal_axis = True, set_ax_limits = True):
    """
    Draws a camera consisting of arrows into a 3d Plot
    Args:   ax            axes object, creates as follows:  fig = plt.figure()
                                                            ax = fig.add_subplot(projection='3d')
            position      np.array(3,) containing the camera position
            direction     np.array(3,3) where each column corresponds to the [x, y, z] axis direction
            length_scale  length scale: the arrows are drawn with length: length_scale * direction
            head_size     controls the size of the head of the arrows
            equal_axis    boolean, if set to True (default) the axis are set to an  equal aspect ratio
            set_ax_limits if set to false, the plot box is not touched by the function
    """
    arrow_prop_dict = dict(mutation_scale=head_size, arrowstyle='-|>', color='r')
    a = Arrow3D([position[0], position[0] + length_scale * direction[0, 0]],
                [position[1], position[1] + length_scale * direction[1, 0]],
                [position[2], position[2] + length_scale * direction[2, 0]],
                **arrow_prop_dict)
    ax.add_artist(a)
    arrow_prop_dict = dict(mutation_scale=head_size, arrowstyle='-|>', color='g')
    a = Arrow3D([position[0], position[0] + length_scale * direction[0, 1]],
                [position[1], position[1] + length_scale * direction[1, 1]],
                [position[2], position[2] + length_scale * direction[2, 1]],
                **arrow_prop_dict)
    ax.add_artist(a)
    arrow_prop_dict = dict(mutation_scale=head_size, arrowstyle='-|>', color='b')
    a = Arrow3D([position[0], position[0] + length_scale * direction[0, 2]],
                [position[1], position[1] + length_scale * direction[1, 2]],
                [position[2], position[2] + length_scale * direction[2, 2]],
                **arrow_prop_dict)
    ax.add_artist(a)

    if not set_ax_limits:
        return

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    zlim = ax.get_zlim()
    ax.set_xlim([min(xlim[0], position[0]), max(xlim[1], position[0])])
    ax.set_ylim([min(ylim[0], position[1]), max(ylim[1], position[1])])
    ax.set_zlim([min(zlim[0], position[2]), max(zlim[1], position[2])])
    
    # This sets the aspect ratio to 'equal'
    if equal_axis:
        ax.set_box_aspect((np.ptp(ax.get_xlim()),
                       np.ptp(ax.get_ylim()),
                       np.ptp(ax.get_zlim())))


def plotTrajectory3D(fps, filename, translations, quaternions, pts3d):
    """
    Draw the trajectory of the camera (3 colored axes, RGB).
    
    Args:   fps           framerate of the video
            filename      filename of the video file to be created
            transl(N, 3):  translations
            quats(N, 4):   orientations given by quaternions [x, y, z, w]
            pts3d(N, 3):   additional 3D points to plot
    
    translation and quaterionss refer to the tranformation T_W_C that maps points from the camera coordinate frame to the world 
    frame, i.e. the transformation that expresses the camera position in the world frame.
    """    
    xmin = min(np.min(translations[:,0]), np.min(pts3d[:,0]))
    xmax = max(np.max(translations[:,0]), np.max(pts3d[:,0]))
    ymin = min(np.min(translations[:,1]), np.min(pts3d[:,1]))
    ymax = max(np.max(translations[:,1]), np.max(pts3d[:,1]))
    zmin = min(np.min(translations[:,2]), np.min(pts3d[:,2]))
    zmax = max(np.max(translations[:,2]), np.max(pts3d[:,2]))

    video = None
    
    fig = plt.figure()
    for i in range(translations.shape[0]):
        fig.clear()
        ax = fig.add_subplot(projection='3d')
        ax.set_xlim([xmin, xmax])
        ax.set_ylim([ymin, ymax])
        ax.set_zlim([zmin, zmax])
    
        ax.set_box_aspect((np.ptp(ax.get_xlim()),
                       np.ptp(ax.get_ylim()),
                       np.ptp(ax.get_zlim())))
        ax.scatter(pts3d[:,0], pts3d[:,1], pts3d[:,2])
        rotMat = Rotation.from_quat(quaternions[i, :]).as_matrix()
        drawCamera(ax, translations[i,:], rotMat, length_scale = 0.1, head_size = 10,
                set_ax_limits = False)

        canvas = FigureCanvas(fig)
        canvas.draw()

        mat = np.array(canvas.renderer._renderer)
        mat = cv2.cvtColor(mat, cv2.COLOR_RGB2BGR)

        if video is None:
            video_size = (mat.shape[1], mat.shape[0])
            video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc('X','V','I','D'), 
                30, video_size)

        mat = cv2.resize(mat, video_size)
        cv2.imshow("test", mat)
        cv2.waitKey(30)
        video.write(mat)

    cv2.destroyAllWindows()
    video.release()

In [20]:
# Load - an undistorted image
#      - the camera matrix
#      - detected corners
image_idx = 1
undist_img_path = "./data/images_undistorted/img_%04d.jpg" % image_idx
undist_img = cv2.imread(undist_img_path, cv2.IMREAD_GRAYSCALE)

K = np.loadtxt("./data/K.txt")
p_W_corners = 0.01 * np.loadtxt("./data/p_W_corners.txt", delimiter = ",")
num_corners = p_W_corners.shape[0]

# Load the 2D projected points that have been detected on the undistorted image into an array
pts_2d = np.reshape( np.loadtxt("./data/detected_corners.txt")[image_idx-1,:], (-1, 2) )

# Now that we have the 2D <-> 3D correspondances let's find the camera pose with respect to the world using the DLT algorithm
M_tilde_dst = estimatePoseDLT(pts_2d, p_W_corners, K)

# Plot the original 2D points and the reprojected points on the image
p_reproj = reprojectPoints(p_W_corners, M_tilde_dst, K)

plt.figure()
plt.imshow(undist_img, cmap = "gray")
plt.scatter(pts_2d[:,0], pts_2d[:,1], marker = 'o')
plt.scatter(p_reproj[:,0], p_reproj[:,1], marker = '+')

In [21]:
# Make a 3D plot containing the corner positions and a visualization of the camera axis
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(p_W_corners[:,0], p_W_corners[:,1], p_W_corners[:,2])

# Position of the camera given in the world frame
R_C_W = M_tilde_dst[:3,:3];
t_C_W = M_tilde_dst[:3,3];
rotMat = R_C_W.T;
pos = -R_C_W.T @ t_C_W;

drawCamera(ax, pos, rotMat, length_scale = 0.1, head_size = 10)
plt.show()

/tmp/ipykernel_25168/1959969964.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Error in callback <function _draw_all_if_interactive at 0x7f143e1a2a60> (for post_execute):


AttributeError: 'RendererAgg' object has no attribute 'M'

In [ ]:
# main video
K = np.loadtxt("./data/K.txt")
p_W_corners = 0.01 * np.loadtxt("../data/p_W_corners.txt", delimiter = ",")
num_corners = p_W_corners.shape[0]

all_pts_2d = np.loadtxt("./data/detected_corners.txt")
num_images = all_pts_2d.shape[0]
translations = np.zeros((num_images, 3))
quaternions = np.zeros((num_images, 4))

for idx in range(num_images):
    pts_2d = np.reshape(all_pts_2d[idx, :], (-1, 2))
    M_tilde_dst = estimatePoseDLT(pts_2d, p_W_corners, K)
    
    R_C_W = M_tilde_dst[:3,:3];
    t_C_W = M_tilde_dst[:3,3];
    quaternions[idx, :] = Rotation.from_matrix(R_C_W.T).as_quat()
    translations[idx,:] = -R_C_W.T @ t_C_W;

fps = 30
filename = "./motion.avi"
plotTrajectory3D(fps, filename, translations, quaternions, p_W_corners)
